<a href="https://colab.research.google.com/github/DonDigidon9/Project/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

code

In [ ]:
from fastapi import FastAPI, HTTPException, Depends, Query
from fastapi.responses import XMLResponse
from databases import Database
from fastapi import FastAPI, HTTPException, Depends, Query, responses


# Создаем экземпляр FastAPI
app = FastAPI()

# Подключение к базе данных SQLite
DATABASE_URL = "sqlite:///./test.db"
database = Database(DATABASE_URL)

# Модель файла
class File:
    def init(self, uuid, virtual_path, created_at, owner_user, owner_group, mode):
        self.uuid = uuid
        self.virtual_path = virtual_path
        self.created_at = created_at
        self.owner_user = owner_user
        self.owner_group = owner_group
        self.mode = mode

# Метод для получения экземпляра базы данных
async def get_database():
    await database.connect()
    yield database
    await database.disconnect()

# Метод для выполнения запросов к базе данных
async def execute_query(query, database=Depends(get_database)):
    return await database.execute(query)

# Метод для получения файла по UUID
async def get_file_by_uuid(uuid: str, database=Depends(get_database)):
    query = "SELECT * FROM files WHERE uuid = :uuid"
    return await database.fetch_one(query=query, values={"uuid": uuid})

# Метод для обработки GET-запроса
@app.get("/file/{uuid}", response_class=XMLResponse)
async def read_file(uuid: str):
    file = await get_file_by_uuid(uuid)
    if file:
        # Возвращаем информацию о файле в формате XML
        return XMLResponse(content=f"<file><uuid>{file['uuid']}</uuid><path>{file['virtual_path']}</path></file>")
    else:
        # Если файл не найден, возвращаем ошибку 404
        raise HTTPException(status_code=404, detail="File not found")

# Пример для PUT-запроса
@app.put("/file/{uuid}")
async def update_file(uuid: str, new_virtual_path: str = Query(..., title="New Virtual Path")):
    file = await get_file_by_uuid(uuid)
    if file:
        # Обновляем виртуальный путь файла в базе данных
        update_query = "UPDATE files SET virtual_path = :new_virtual_path WHERE uuid = :uuid"
        await execute_query(update_query, values={"new_virtual_path": new_virtual_path, "uuid": uuid})
        return {"message": "File updated successfully"}
    else:
        raise HTTPException(status_code=404, detail="File not found")

#  Пример для HEAD-запроса
@app.head("/file/{uuid}")
async def head_file(uuid: str):
    file = await get_file_by_uuid(uuid)
    if file:
        # Возвращаем только заголовки без тела ответа
        headers = {"Content-Length": str(len(file['virtual_path']))}
        return XMLResponse(content="", headers=headers)
    else:
        raise HTTPException(status_code=404, detail="File not found")

# Пример для OPTIONS-запроса
@app.options("/file/{uuid}")
async def options_file(uuid: str):
    # Возвращаем разрешенные методы и другие заголовки в ответе
    allowed_methods = ["GET", "PUT", "HEAD", "OPTIONS", "PROPFIND", "MOVE", "COPY", "DELETE", "LOCK", "UNLOCK"]
    headers = {"Allow": ", ".join(allowed_methods)}
    return XMLResponse(content="", headers=headers)

# Пример для PROPFIND-запроса
@app.propfind("/file/{uuid}")
async def propfind_file(uuid: str):
    file = await get_file_by_uuid(uuid)
    if file:
        # Возвращаем информацию о файле в формате XML
        return XMLResponse(content=f"<file><uuid>{file['uuid']}</uuid><path>{file['virtual_path']}</path></file>")
    else:
        raise HTTPException(status_code=404, detail="File not found")

# Пример для MOVE-запроса
@app.move("/file/{uuid}")
async def move_file(uuid: str, new_virtual_path: str = Query(..., title="New Virtual Path")):
    file = await get_file_by_uuid(uuid)
    if file:
        # Обновляем виртуальный путь файла в базе данных
        update_query = "UPDATE files SET virtual_path = :new_virtual_path WHERE uuid = :uuid"
        await execute_query(update_query, values={"new_virtual_path": new_virtual_path, "uuid": uuid})
        return {"message": "File moved successfully"}
    else:
        raise HTTPException(status_code=404, detail="File not found")

# Пример для COPY-запроса
@app.copy("/file/{uuid}")
async def copy_file(uuid: str, new_uuid: str = Query(..., title="New UUID")):
    file = await get_file_by_uuid(uuid)
    if file:
        # Создаем копию файла с новым UUID
        copy_query = "INSERT INTO files (uuid, virtual_path, created_at, owner_user, owner_group, mode) VALUES (:new_uuid, :virtual_path, :created_at, :owner_user, :owner_group, :mode)"
        await execute_query(copy_query, values={"new_uuid": new_uuid, **file})
        return {"message": "File copied successfully"}
    else:
        raise HTTPException(status_code=404, detail="File not found")

# Пример для DELETE-запроса
@app.delete("/file/{uuid}")
async def delete_file(uuid: str):
    file = await get_file_by_uuid(uuid)
    if file:
        # Удаляем файл из базы данных
        delete_query = "DELETE FROM files WHERE uuid = :uuid"
        await execute_query(delete_query, values={"uuid": uuid})
        return {"message": "File deleted successfully"}
    else:
        raise HTTPException(status_code=404, detail="File not found")

# Пример для LOCK-запроса
@app.lock("/file/{uuid}")
async def lock_file(uuid: str):
    # Реализуйте логику блокировки файла здесь
    return {"message": "File locked successfully"}

# Пример для UNLOCK-запроса
@app.unlock("/file/{uuid}")
async def unlock_file(uuid: str):
    # Реализуйте логику разблокировки файла здесь
    return {"message": "File unlocked successfully"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)